# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [ ]:
import os, re, time, copy
import html, hashlib, itertools, operator
from collections import defaultdict, deque
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [ ]:
class VirtualMachine:
    MODULUS = 32768

    def __init__(self, program_bytes: bytes, debug: bool = False):
        """Initialize the Virtual Machine."""
        # Core configuration
        self.pointer = 0                      # Instruction pointer
        self.debug = debug                    # Enables debug logging
        self.paused = False                   # Indicates if execution is paused (waiting for input)
        self.running = True                   # Indicates if the VM is running
        self.version_no = 0

        # State components
        self.memory = [0] * self.MODULUS      # Memory: 15-bit address space 32K words of 16-bit values
        self.registers    = [0] * 8           # 8 general-purpose registers
        self.output_stack = [ ]               # Unbounded stack for temporary values
        self.output_terminal = [""]           # Output buffer for character writes
        self.input_commands  = []             # Input buffer for keyboard simulation
        self.op_log = []                      # Log of executed operations (used for debugging/tracing)
        self.program_bytes = program_bytes    # Create an initial copy of the bytes program

        self.program = [                      # Convert binary program to list of integers
            (high << 8) | low
            for low, high in zip(program_bytes[::2], program_bytes[1::2])
        ]
        self.memory[:len(self.program)] = self.program

        self.opcode_map = {
            0:  (self.__terminate, 0),
            1:  (self.__set,  2),
            2:  (self.__push, 1),
            3:  (self.__pop,  1),
            4:  (self.__equal_to, 3),
            5:  (self.__greater,  3),
            6:  (self.__jump,     1),
            7:  (self.__jump_to,  2),
            8:  (self.__jump_if,  2),
            9:  (self.__arithmetic, (3, "add")),
            10: (self.__arithmetic, (3, "mul")),
            11: (self.__arithmetic, (3, "mod")),
            12: (self.__and_or,     (3, "and")),
            13: (self.__and_or,     (3, "or")),
            14: (self.__not,   2),
            15: (self.__rmem,  2),
            16: (self.__wmem,  2),
            17: (self.__call,  1),
            18: (self.__ret,   0),
            19: (self.__out,   1),
            20: (self.__input, 1),
            21: (self.__noop,  0)
        }

    def __validate_val(self, val):
        if 0 <= val <= 32767:
            return val
        else:
            return self.registers[self.__get_reg_no(val)]

    def __get_reg_no(self, val):
        if 32768 <= val <= 32775:
            return val - self.MODULUS
        raise ValueError(f"Expected register (32768-32775), got {val}")

    def __get_args(self, total_args):
        """Retrieve a list of arguments from memory starting from the current pointer position."""
        return self.memory[self.pointer + 1 : self.pointer + 1 + total_args]

    def __set_registers(self, addr, val):
        """Set register[`addr`] to desired `value`"""
        self.registers[addr] = val

    def __terminate(self, args_count):
        """[00]: Stop Execution and Terminate the program. """
        self.debug_log(f"[{self.pointer:05}: HALT] __PROGRAM TERMINATED__")
        self.running = False
        return args_count

    def __set(self, args_count):
        """[01]: Set register `a` to the value of `b`."""
        a_raw, b_raw = self.__get_args(args_count)
        a, b = self.__get_reg_no(a_raw), self.__validate_val(b_raw)
        self.__set_registers(a, b)
        self.debug_log(f"[{self.pointer:05}: SET] reg[{a}] = {b}")
        return args_count

    def __push(self, args_count):
        """[02]: Push `a` onto the stack."""
        a_raw, = self.__get_args(args_count)
        a = self.__validate_val(a_raw)
        self.output_stack.append(a)
        self.debug_log(f"[{self.pointer:05}: PSH] Pushed Value({a}) onto stack {a_raw} {a}")
        return args_count

    def __pop(self, args_count):
        """[03]: Pop from the stack into register `a`. Raises error if empty."""
        if not self.output_stack:
            raise RuntimeError("Stack is empty")
        a_raw, = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        pop_val = self.output_stack.pop()
        self.__set_registers(a, pop_val)
        self.debug_log(f"[{self.pointer:05}: POP] Pop {pop_val} from stack onto reg[{a}]")
        return args_count

    def __equal_to(self, args_count):
        """[04]: Set `a` to 1 if `b == c`, else 0."""
        a_raw, b_raw, c_raw = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        b, c = [self.__validate_val(i) for i in (b_raw, c_raw)]
        val = int(b == c)
        condition = "==" if val else "!="
        self.__set_registers(a, val)
        self.debug_log(f"[{self.pointer:05}: EQL] reg[{a}] = {val} (cond: {b} {condition} {c})")
        return args_count

    def __greater(self, args_count):
        """[05]: Set `a` to 1 if `b > c`, else 0."""
        a_raw, b_raw, c_raw = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        b, c = [self.__validate_val(i) for i in (b_raw, c_raw)]
        val = int(b > c)
        self.__set_registers(a, val)
        condition = ">" if val else "<="
        self.debug_log(f"[{self.pointer:05}: GRT] reg[{a}] = {val} (cond: {b} {condition} {c})")
        return args_count

    def __jump(self, args_count):
        """[06]: Jump to address `a`."""
        a_raw, = self.__get_args(args_count)
        a = self.__validate_val(a_raw)
        self.debug_log(f"[{self.pointer:05}: JMP] Jump pointer to {a:05}")
        self.pointer = a

    def __jump_to(self, args_count):
        """[07]: Jump to `b` if `a` != 0, else proceed."""
        raw_args = self.__get_args(args_count)
        a, b = [self.__validate_val(raw) for raw in raw_args]
        jump = b if (a != 0) else (self.pointer + args_count +1)
        condition = "!=" if a != 0 else "=="
        self.debug_log(f"[{self.pointer:05}: JNZ] Jump pointer to {jump:05} (cond: {a} {condition} 0)")
        self.pointer = jump

    def __jump_if(self, args_count):
        """[08]: Jump to `b` if `a` == 0, else proceed."""
        raw_args = self.__get_args(args_count)
        a, b = [self.__validate_val(raw) for raw in raw_args]
        jump = b if (a == 0) else (self.pointer + args_count +1)
        condition = "==" if (a == 0) else "!="
        self.debug_log(f"[{self.pointer:05}: JFZ] Jump pointer to {jump:05} (cond: {a} {condition} 0)")
        self.pointer = jump

    def __arithmetic(self, func_args):
        """[09, 10, 11]: Aritmethic Operation `add`, `mul`, and `mod`"""
        args_count, op_type = func_args
        op_sign = {'add':'+', 'mul':'x', 'mod':'%'}[op_type]
        a_raw, b_raw, c_raw = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        b, c = [self.__validate_val(i) for i in (b_raw, c_raw)]
        if op_type == "add":
            val = (b + c) % self.MODULUS
        elif op_type == "mul":
            val = (b * c) % self.MODULUS
        elif op_type == "mod":
            val = (b % c)
        self.__set_registers(a, val)
        info = f"[{self.pointer:05}: {op_type.upper()}] reg[{a}] = {val} (eq: {b} {op_sign} {c})"
        self.debug_log(info)
        return args_count

    def __and_or(self, func_args):
        """[12, 13]: Bitwise `and`|`or` operations for values `b` and `c`"""
        args_count, op_type = func_args
        op_sign = {'and':'&', 'or':'|'}[op_type]
        a_raw, b_raw, c_raw = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        b, c = [self.__validate_val(i) for i in (b_raw, c_raw)]
        if op_type == "and":
            val = b & c
        elif op_type == "or":
            val = b | c
            op_type += " "
        self.__set_registers(a, val)
        self.debug_log(f"[{self.pointer:05}: {op_type.upper()}] reg[{a}] = {val} (bit_op: {b} {op_sign} {c})")
        return args_count

    def __not(self, args_count):
        """[14]: Stores 15-bit bitwise inverse of `b` in `a`"""
        a_raw, b_raw = self.__get_args(args_count)
        a, b = self.__get_reg_no(a_raw), self.__validate_val(b_raw)
        inv_bits = (~b) & (self.MODULUS - 1)       # Invert and mask to 15 bits
        self.__set_registers(a, inv_bits)
        self.debug_log(f"[{self.pointer:05}: NOT] reg[{a}] = {inv_bits} (15 bit inverse of {b} stored)")
        return args_count

    def __rmem(self, args_count):
        """[15]: Read Memory at address `b` and write it `a`"""
        a_raw, b_raw = self.__get_args(args_count)
        a, b = self.__get_reg_no(a_raw), self.__validate_val(b_raw)
        val = self.memory[b]
        self.__set_registers(a, val)
        self.debug_log(f"[{self.pointer:05}: RDM] reg[{a}] = {val}")
        return args_count

    def __wmem(self, args_count):
        """[16]: Write val from `b` into memory address at `a`"""
        raw_args = self.__get_args(args_count)
        a, b = [self.__validate_val(raw) for raw in raw_args]
        self.memory[a] = b
        self.debug_log(f"[{self.pointer:05}: WRM] Write to memory at a({a}) with value {b}")
        return args_count

    def __call(self, args_count):
        """[17]: Write the address of the next instruction to the stack and jump to `a`"""
        a_raw, = self.__get_args(args_count)
        a = self.__validate_val(a_raw)
        stack_val = self.pointer + args_count + 1
        self.output_stack.append(stack_val)
        self.debug_log(f"[{self.pointer:05}: CAL] Call to {a}, return to {stack_val}")
        self.pointer = a

    def __ret(self, args_count):
        """[18]: Remove the top element from the stack and jump to it; empty stack = halt"""
        if not self.output_stack:
            self.debug_log(f"[{self.pointer:05}: RET]  EMPTY OUTPUT STACK")
            self.__terminate(args_count)
        ret_addr = self.output_stack.pop()
        self.debug_log(f"[{self.pointer:05}: RET] Return to {ret_addr}")
        self.pointer = ret_addr

    def __out(self, args_count):
        """[19]: Write the character represented by ascii code `a` to the terminal"""
        a_raw,  = self.__get_args(args_count)
        a = self.__validate_val(a_raw)
        char_a = chr(a)
        self.output_terminal[-1] += char_a
        self.debug_log(f"[{self.pointer:05}: OUT]{repr(char_a)} to terminal (ASCII Code: {a})")
        return args_count

    def __input(self, args_count):
        """[20]: Read a character from the terminal and write its ascii code to `a`"""
        if not self.input_commands:
            self.paused = True
            self.debug_log(f"[{self.pointer:05}: INP] Empty Input | Pause Computer)")
            return None
        a_raw,  = self.__get_args(args_count)
        a = self.__get_reg_no(a_raw)
        input_char = self.input_commands.pop(0)
        input_ascii = ord(input_char)
        self.__set_registers(a, input_ascii)
        self.debug_log(f"[{self.pointer:05}: INP] reg[{a}] = {input_ascii} ({input_char} -> {input_ascii})")
        return args_count

    def __noop(self, args_count):
        """[21]: No Operation"""
        self.debug_log(f"[{self.pointer:05}: NOP] No Operation Performed")
        return args_count

    def replicate(self, copy_no = 1):
        """Create and return a copy of the current VirtualMachine instance, preserving its state."""
        # Create a new instance with the original program bytes
        new_vm = VirtualMachine(self.program_bytes, debug=self.debug)

        # Deep copy the current VM state
        new_vm.paused = self.paused
        new_vm.running = self.running
        new_vm.pointer = self.pointer
        new_vm.memory  = self.memory.copy()
        new_vm.registers = self.registers.copy()
        new_vm.output_stack = self.output_stack.copy()
        new_vm.input_commands = self.input_commands.copy()
        new_vm.output_terminal = self.output_terminal.copy()
        new_vm.version_no = copy_no

        # Copy operation log and add replication marker
        new_vm.op_log = self.op_log.copy()
        new_vm.debug_log(f"[{new_vm.version_no:05}_COPY] __REPLICATED VM AT CURRENT STATE__")

        return new_vm

    def add_input(self, input_commands):
        """Add input commands to the VM's input commands queue."""
        if isinstance(input_commands, list):
            input_commands = '\n'.join(input_commands)
        else:
            input_commands = [input_commands]

        self.input_commands.extend(input_commands)
        self.output_terminal.append("")
        self.paused = False
        self.debug_log(f"[ADD INPUTS] Add series of commands to input commands queue")

    def debug_log(self, message):
        """If debug is True, add op_message to an operation log"""
        if self.debug:
            self.op_log.append(message)

    def save_log(self, file_name = "vm_log", file_loc = ""):
        """Save log to a .txt file"""
        import os
        full_path = os.path.join(file_loc, f"{file_name}.txt")
        with open(full_path, "w") as f:
            f.write('\n'.join(self.op_log))

    def monkey_patching(self, patches):
        self.patched = patches

    def run_computer(self, input_commands = []):
        """Run Computer with an initial list of commands"""
        if input_commands:
            self.add_input(input_commands + [""])

        while self.running and not self.paused:
            opcode = self.memory[self.pointer]
            if opcode not in self.opcode_map:
                raise ValueError(f"Unknown opcode {opcode} at position {self.pointer}")

            # Call monkey patch if available
            if hasattr(self, "patched") and callable(self.patched):
                self.pointer, self.registers, self.memory = self.patched(self.pointer, self.registers, self.memory)

            op_fn, (func_args) = self.opcode_map[opcode]
            move_ip = op_fn(func_args)

            if move_ip is not None:
                self.pointer += move_ip + 1

        return self.output_terminal


In [ ]:
# Load the binary program file
program_file = "challenge1.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [ ]:
class SynacorConsole:

    def __init__(self, software: bytes, spec_code: str, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.spec_code = spec_code
        self.start_time = time.time()
        self.console = self.reset_console()

    def reset_console(self):
        """Reset console state and reinitialize the virtual machine."""
        console = VirtualMachine(self.software)
        self.game_map = defaultdict(list)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.code_no = 0
        self.start_time = time.time()
        self.prev_time = self.start_time
        self.__extract_codes(" ", [self.spec_code])  # Preload spec code
        return console

    def __extract_codes(self, text: str, codes: list[str] = []) -> list[str]:
        """
        Extract valid mixed-case challenge codes from the given text
        and record their MD5 hashes with timestamps.
        """
        # Pattern: must include at least one lowercase followed by uppercase and then lowercase again
        found = re.findall(r"[A-Za-z]*[a-z]+[A-Z]+[a-z]+[A-Za-z]*", text)
        codes.extend(found)

        new_codes = []
        current_time = time.time()

        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                total_time = f"{current_time - self.start_time:.5f}s"
                code_time = f"{current_time - self.prev_time:.5f}s"
                self.challenge_codes[self.code_no] = (
                    code, self.md5_hash(code), code_time, total_time,
                )
                new_codes.append(code)
                self.valid_codes.add(code)
                self.prev_time = current_time

        return new_codes

    @staticmethod
    def md5_hash(code):
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    @staticmethod
    def parse_equation(eq_str):
        lhs, rhs = eq_str.split('=')
        rhs = int(rhs.strip())
        # Replace underscores with placeholder p[i]
        parts = re.split(r'(_)', lhs.strip())
        indices = [i for i, x in enumerate(parts) if x == '_']

        def equation(p):
            expr = parts[:]
            for i, idx in enumerate(indices):
                expr[idx] = str(p[i])
            expr_str = ''.join(expr).replace('^', '**')

            # Tokenize
            tokens = re.findall(r'\d+|[\+\-\*]{1,2}', expr_str)

            # Evaluate with operator precedence: ** > * > +/-
            def apply(ops, precedence):
                stack = []
                i = 0
                while i < len(ops):
                    if ops[i] in precedence:
                        a, b = int(stack.pop()), int(ops[i+1])
                        op_func = {'+': operator.add, '-': operator.sub,
                                    '*': operator.mul, '**': operator.pow}[ops[i]]
                        stack.append(str(op_func(a, b)))
                        i += 2
                    else:
                        stack.append(ops[i])
                        i += 1
                return stack
            # Handle ** first, then *, then +/-
            for prec in (['**'], ['*'], ['+', '-']):
                tokens = apply(tokens, prec)

            return int(tokens[0]) == rhs

        return equation

    def __format_terminal(self, text, code_color, input_color):
        """Highlight any detected code-like strings in the terminal."""
        lines = []
        for line in text.splitlines():
            # Simple input highlight: lines starting with '>'
            if line.strip().startswith('>>'):
                line = f"<span style='color:{input_color}'>{line}</span>"
            else:
                line = re.sub(      # Example: highlight code-like patterns
                    r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
                    rf"<span style='color:{code_color}'>\1</span>", line
                )
            lines.append(line)
        return "\n".join(lines)

    def display_terminal(self, terminal_text, actions = []):
        """Format and display terminal output with highlighted codes and interleaved actions."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00",
            "codes": "#FF0000", "input": "#0073FF"
        }
        html_valid_text = html.escape(terminal_text)
        # Normalize line endings and remove excessive empty lines
        html_valid_text = re.sub(r'\n{3,}', '\n\n', html_valid_text.strip())  # At most 2 newlines
        parts = re.split(r'(What do you do\?)', html_valid_text)

        output_lines = []

        if actions:
            output_lines.append(f">> {actions[0]}")
        action_index = 1 if actions else 0

        # Now interleave remaining actions after each prompt
        i = 0
        while i < len(parts):
            output_lines.append(parts[i])
            input_prompt = "What do you do?"
            if i + 1 < len(parts) and parts[i + 1] == input_prompt:
                output_lines.append(input_prompt) # Append prompt
                if action_index < len(actions):   # Append next action if available
                    output_lines.append(f">> {actions[action_index]}")
                    action_index += 1
                i += 2
            else:
                i += 1

        combined_text = "\n".join(output_lines)
        longest_line = max(max(len(line) for line in combined_text.split('\n')), 150)

        html_formatted = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(combined_text, self.color_dict['codes'], self.color_dict['input'])
            + "</pre></div>"
        )

        display(HTML(html_formatted))

    def play_game_manually(self, actions = []):
        """
        Game Instructions:
        - `look`: You may merely 'look' to examine the room, or you may 'look ' (such as 'look chair') to examine something specific.
        - `go`: You may 'go ' to travel in that direction (such as 'go west'), or you may merely '' (such as 'west').
        - `inv`: To see the contents of your inventory, merely 'inv'.
        - `take`: You may 'take ' (such as 'take large rock').
        - `drop`: To drop something in your inventory, you may 'drop '.
        - `use`: You may activate or otherwise apply an item with 'use '.
        """
        if actions:
            self.overall_commands.extend(actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.__extract_codes(current_terminal)
        if self.visualize:
            self.display_terminal(current_terminal, actions)
        return self.overall_commands, self.challenge_codes

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        location, description, things, exits = ("", "", [], [])

        current_section = None
        for line_no, line in enumerate(lines):
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
                description = lines[line_no + 1]
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, description, things, exits

    def create_room_id(self, room_description):
        h = self.md5_hash(room_description.strip())
        # Take first 4 hex digits, convert to int, and mod 10000 to get a 4-digit number
        return int(h[:8], 16) % 10000

    def __use_items(self, item):
        """Return actions to collect and use the item, and what items will be added to inventory."""
        if item == "empty lantern":
            return [f"take {item}", "use can", "use lantern"], {"lit lantern"}
        elif item in ["strange book", "business card"]:
            return [f"take {item}", f"look {item}",], {item}
        elif "coin" in item:
            return [f"take {item}"], {item}
        else:
            return [f"take {item}", f"use {item}"], {item}

    def __solve_coins_puzzle(self, inventory, game_copy, equation):
        word_to_num = {
            "two": 2, "three": 3, "four": 4, "five": 5,
            "six": 6, "seven": 7, "eight": 8, "nine": 9,
            "digon": 2, "triangle": 3, "square": 4, "pentagon": 5,
            "hexagon": 6, "heptagon": 7, "octagon": 8, "nonagon": 9,
        }

        coin_items = [item for item in inventory if "coin" in item]
        look_coins = [f"look {item}" for item in coin_items]

        coin_game = game_copy.replicate()
        *_, final_terminal = coin_game.run_computer(look_coins)
        coin_lines = [line for line in final_terminal.splitlines() if "coin" in line]

        # Build coin_dict: shape/number -> coin item name
        coin_dict = {}
        for item in coin_items:
            coin_type = item.replace("coin", "").strip()
            coin_type = "rounded" if coin_type == "concave" else coin_type

            matching_line = next((line for line in coin_lines if coin_type in line), "")
            match = next((num for word, num in word_to_num.items() if word in matching_line), None)

            if match:
                coin_dict[match] = item

        # Solve the equation by trying permutations of the coin values
        solution = next(filter(self.parse_equation(equation), itertools.permutations(coin_dict.keys())))

        # Use coins in the determined order
        use_coins = [f"use {coin_dict[num]}" for num in solution]
        return look_coins + use_coins

    def __solve_mystery_puzzle(self, game_copy, action_list):
        manipulate_game = game_copy.replicate()
        # test_console = VirtualMachine(self.software, True)
        # test_console.run_computer(action_list)
        # test_console.save_log()
        M = game_copy.MODULUS
        def patches(ip):
            if ip == 5451:
                manipulate_game.registers[7] = 25734
            elif ip == 5489:
                manipulate_game.memory[5489] = manipulate_game.memory[5490] = 21
                manipulate_game.registers[0] = 6

        print(game_copy.pointer)
        return []

    def bfs_exploration(self):
        """Perform BFS to explore and map out the game world."""
        collect_coins = {'concave coin', 'shiny coin', 'red coin', 'blue coin', 'corroded coin'}
        coins_puzzle, mystery_puzzle, grid_puzzle = (False, False, False)
        visited, bfs_history = (set(), defaultdict(tuple))
        steps, MAX_STEPS = (0, 750)
        complete_solution = []

        # Each entry: (console_state, pending_actions, path_so_far, action_history)
        # Use deque for efficient popping from the left
        queue = deque([(self.console, [], [], set())])

        while queue and steps < MAX_STEPS:
            state, pending_actions, action_history, game_inv = queue.popleft()
            future_actions = []
            steps += 1
            if "use teleporter" in action_history:
                complete_solution = action_history[:]

            # Run actions on a fresh copy of the game state
            game_copy = state.replicate(steps)
            *_, last_terminal = game_copy.run_computer(pending_actions)
            bfs_history[steps] = (game_copy, action_history)
            self.__extract_codes(last_terminal)

            # Parse game state
            room, purpose, all_items, room_exits = self.__parse_game_state(last_terminal)
            room_id = self.create_room_id(purpose)

            # Skip if already visited with current inventory
            if (room_id, tuple(game_inv)) in visited:
                continue
            visited.add((room_id, tuple(game_inv)))

            # Clone game_inv for branching paths
            base_inv = game_inv.copy()

            if (
                len(collect_coins & game_inv) == 5 and not coins_puzzle and
                'You stand in the massive central hall of these ruins.' in purpose
                ):
                equation = next((line.strip() for line in last_terminal.splitlines() if " = " in line), None)
                coin_solution = self.__solve_coins_puzzle(game_inv, game_copy, equation)
                coins_puzzle = True
                action_history.extend(coin_solution)
                future_actions.extend(coin_solution)
                base_inv = {"all coins used", "lit lantern", "tablet"}
                queue.append((game_copy, coin_solution, action_history, base_inv))

            if "look strange book" in pending_actions and not mystery_puzzle:
                # self.display_terminal(last_terminal, pending_actions)
                reset_actions = self.__solve_mystery_puzzle(game_copy, action_history)
                mystery_puzzle = True

            # Handle item collection
            for item in all_items:
                # Skip collecting lantern unless can is collected
                if (item == "empty lantern") and ("can" not in game_inv):
                    continue
                item_actions, collected = self.__use_items(item)

                base_inv.update(collected)
                future_actions.extend(item_actions)
                action_history.extend(item_actions)
                queue.append((game_copy, future_actions, action_history, base_inv))

            # Explore exits
            for direction in room_exits:
                is_dark_passage = (purpose == "You are in a dark, narrow passage.")

                # If override: move twice in the same direction
                dir_sequence = [direction, direction] if is_dark_passage else [direction]

                next_actions = future_actions + dir_sequence
                next_history = action_history + dir_sequence
                queue.append((game_copy, next_actions, next_history, base_inv))

        print("Steps:",steps)
        return complete_solution, self.challenge_codes

    def auto_play(self):
        """Automatic playthrough using the bfs exploration."""
        game_commands, bfs_times = self.bfs_exploration()
        full_game_run = self.reset_console()
        init_terminal = full_game_run.run_computer()
        self.display_terminal(init_terminal[-1])

        *_, full_terminal = full_game_run.run_computer(game_commands)
        self.display_terminal(full_terminal, game_commands)
        self.__extract_codes(full_terminal)
        self.overall_commands = game_commands.copy()

        return self.overall_commands, bfs_times

In [ ]:
console = SynacorConsole(vm_program, "LDOb7UGhTi")
# # command_list, challenge_codes = console.auto_play()
# command_list, challenge_codes = console.bfs_exploration()

In [ ]:
manual_console = SynacorConsole(vm_program, "LDOb7UGhTi", False)
manual_console.play_game_manually()
game_actions = [
    'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west',
    'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern',
    'east', 'east', 'take empty lantern', 'use can', 'use lantern', 'west', 'west', 'passage', 'darkness',
    'continue', 'west', 'west', 'west', 'west', 'north', 'take red coin', 'north', 'east', 'take concave coin',
    'down', 'take corroded coin', 'up', 'west', 'west', 'take blue coin', 'up', 'take shiny coin', 'down',
    'east', 'look corroded coin', 'look concave coin', 'look red coin', 'look shiny coin', 'look blue coin',
    'use blue coin', 'use red coin', 'use shiny coin', 'use concave coin', 'use corroded coin', 'north',
    'take teleporter', 'use teleporter', 'take business card', 'look business card', 'inv',
    ]
all_actions, challenge_codes = manual_console.play_game_manually(game_actions)

def software_patch(pointer, registers, memory):
    if pointer == 5451:
        registers[7] = 25734
    elif pointer == 5489:
        memory[5489] = 21
        memory[5490] = 21
        registers[0] = 6
    return pointer, registers, memory

manual_console.visualize = True
# manual_console.console.monkey_patching(software_patch)
all_actions, challenge_codes = manual_console.play_game_manually(["use teleporter"])


In [ ]:
for code_no, code_props in challenge_codes.items():
    print(f"Code {code_no}: {code_props[0]:12} -> {code_props[1]} |", \
        f"Times: Code = {code_props[2]}, Total = {code_props[3]}")
print(f"Complete Run Time = {time.time() - start_time:.5f}s")

In [ ]:
program = console.console.program.copy()
M = VirtualMachine.MODULUS # modulus

(
    OP_halt,
    OP_set,
    OP_push,
    OP_pop,
    OP_eq,
    OP_gt,
    OP_jmp,
    OP_jt,
    OP_jf,
    OP_add,
    OP_mult,
    OP_mod,
    OP_and,
    OP_or,
    OP_not,
    OP_rmem,
    OP_wmem,
    OP_call,
    OP_ret,
    OP_out,
    OP_in,
    OP_noop
) = range(22)
instructions = [
    # name, argcount
    ("halt", 0),
    ("set",  2),
    ("push", 1),
    ("pop",  1),
    ("eq",   3),
    ("gt",   3),
    ("jmp",  1),
    ("jt",   2),
    ("jf",   2),
    ("add",  3),
    ("mult", 3),
    ("mod",  3),
    ("and",  3),
    ("or",   3),
    ("not",  2),
    ("rmem", 2),
    ("wmem", 2),
    ("call", 1),
    ("ret",  0),
    ("out",  1),
    ("in",   1),
    ("noop", 0)
]

def format_instruction(address, instruction):
    def format_arg(n):
        if instruction[0] == OP_out:
            return repr(chr(n))
        else:
            return str(n) if n < M else f"R{n-M}"
    return (
        f"{address:5}: {instructions[instruction[0]][0]:4} "
        + " ".join(format_arg(n) for n in instruction[1:])
    ).rstrip()

def disassemble(program, start, n):
    print(f'\n {start=}, {n=}')
    # Prints n instructions starting from address `start`.
    i = start
    j = 0
    while j < n:
        argcount = instructions[program[i]][1]
        print(format_instruction(i, program[i:i+1+argcount]))
        i += 1+argcount
        j += 1

# disassemble(program, 5451, 21)
# disassemble(program, 6027, 16)

def A(m, n, k):
    stack = [m, n]
    while len(stack) > 1:
        n, m = stack.pop(), stack.pop()
        if m == 0:
            stack.append((n+1)%M)
        elif m == 1:
            stack.append((n+k+1)%M)
        elif m == 2:
            stack.append(((n+2)*k+n+1)%M)
        elif n == 0:
            stack.extend([m-1, k])
        else:
            stack.extend([m-1, m, n-1])
    return stack[0]

ans = next(
    filter(
        lambda k: A(4, 1, k) == 6,
        itertools.chain(range(25730, 32768), range(1, 25730))
    )
)
# print(ans)